## Analysis of population genetic data in Python with scikit-allel

This tutorial will provide an introduction to the powerful [scikit-allel](https://scikit-allel.readthedocs.io/en/stable/#) package, which we'll use to manipulate and describe our variant (SNP) data. To begin, we'll import it, along with the standard Python library numpy:

In [1]:
import os
import allel
import numpy as np
import pandas as pd

### Basic data structures and functions

Before we use empirical data, let's get a feel for scikit-allel's GenotypeArray objects using a simple example. Here, we set up a GenotypeArray with two individuals (as columns) and three loci (as rows). Each integer in this array refers to an allele, where 0 indicates the reference allele, 1 the first alternate allele, 2 the second, etc. Any negative integer indicates missing data.

In [2]:
g = allel.GenotypeArray([[[0, 0], [0, 0]],
                         [[0, 0], [0, 1]],
                         [[0, 0], [1, 1]],
                         [[0, 1], [1, 1]],
                         [[1, 1], [1, 1]],
                         [[0, 0], [1, 2]],
                         [[0, 1], [1, 2]],
                         [[0, 1], [-1, -1]],
                         [[-1, -1], [-1, -1]]])

This is what that object looks like: 

In [3]:
g

<GenotypeArray shape=(9, 2, 2) dtype=int64>
0/0 0/0
0/0 0/1
0/0 1/1
...
0/1 1/2
0/1 ./.
./. ./.

Our GenotypeArray object "g" has attributes reflecting its dimensions, its number of variants, ploidy and sample size. For example: 

In [4]:
g.ndim

3

In [7]:
g.shape

(9, 2, 2)

In [8]:
g.n_variants

9

In [9]:
g.ploidy

2

In [10]:
g.n_samples

2

With this object, we can begin to get a feel for scikit-allel's functions for describing diversity and divergence. For example, we can calculate observed heterozygosity, generating an array for each locus:

In [7]:
het_obs = allel.heterozygosity_observed(g)

9

Using the `namean()` function in `numpy` to ignore NA values, we can calculate the genome-wide average:

In [8]:
np.nanmean(het_obs)

0.4375

A natural next step is to see whether these freququencies are a deviation from Hardy-Weinberg Equilibrium. To do so, we first calculate allele frequencies for each locus, using the `count_alleles().to_frequencies()` function:

In [9]:
af = g.count_alleles().to_frequencies()

In [10]:
af

array([[1.  , 0.  , 0.  ],
       [0.75, 0.25, 0.  ],
       [0.5 , 0.5 , 0.  ],
       [0.25, 0.75, 0.  ],
       [0.  , 1.  , 0.  ],
       [0.5 , 0.25, 0.25],
       [0.25, 0.5 , 0.25],
       [0.5 , 0.5 , 0.  ],
       [ nan,  nan,  nan]])

In this array, each row is a locus, and the columns 0,1 and 2 refer to reference, 1st alternate, and 2nd alternate alleles. Next, we can use these data to calculate expected heterozygosity:

In [11]:
allel.heterozygosity_expected(af, ploidy=2)

array([0.   , 0.375, 0.5  , 0.375, 0.   , 0.625, 0.625, 0.5  ,   nan])

Looks different to me! 

## Loading and examining variant data

Next up, we're going to pivot to looking at empirical data: specifically, WGS data from across These data are in Variant Call Format, and can be found as `syma.vcf.gz` in the tutorial folder. 

To begin, we'll use scikit-allel to import the .vcf as a `numpy` array. Here, it's important to use the regex `'*'` wildcard, in order to extract all possible data from the file. 

In [12]:
s = allel.read_vcf("syma.tutorial.vcf.gz", fields='*')

The `numpy` array object `r` has a set of keys representing aspects of the .vcf file. We can list these with the `.keys()` function, sorting by name:

In [13]:
sorted(s.keys())

[u'calldata/AD',
 u'calldata/DP',
 u'calldata/GQ',
 u'calldata/GT',
 u'calldata/PL',
 'samples',
 u'variants/AC',
 u'variants/AF',
 u'variants/ALT',
 u'variants/AN',
 u'variants/BaseQRankSum',
 'variants/CHROM',
 u'variants/DP',
 u'variants/DS',
 u'variants/Dels',
 u'variants/ExcessHet',
 u'variants/FILTER_LowQual',
 u'variants/FILTER_PASS',
 u'variants/FS',
 u'variants/HaplotypeScore',
 u'variants/ID',
 u'variants/InbreedingCoeff',
 u'variants/MLEAC',
 u'variants/MLEAF',
 u'variants/MQ',
 u'variants/MQ0',
 u'variants/MQRankSum',
 'variants/POS',
 u'variants/QD',
 u'variants/QUAL',
 'variants/REF',
 u'variants/RPA',
 u'variants/RU',
 u'variants/ReadPosRankSum',
 u'variants/SOR',
 u'variants/STR',
 'variants/altlen',
 'variants/is_snp',
 'variants/numalt']

Here, anything begining with "callset" is from the INFO field of the .vcf, while "variants" refers to data associated with each SNP (or site, if you have invariant alleles in your file). (Some of this information, e.g. depth, is redundant, depending on your SNP caller.) To examine any of these in greater detail, we can select the relevant index. For example, to look at the position of these SNPs (relative to their reference), we can select `variants/POS` key:

In [16]:
s['variants/POS']

120150

Or a list of all sample IDs:

In [17]:
s['samples']

array([u'EL10_toro', u'EL11_toro', u'EL13_toro', u'EL18_mega',
       u'EL19_mega', u'EL1_mega', u'EL20_mega', u'EL21_toro',
       u'EL23_mega', u'EL24_mega', u'EL27_mega', u'EL29_ochr',
       u'EL32_toro', u'EL39_toro', u'EL4_mega', u'EL6_mega', u'EL8_toro',
       u'EL9_toro'], dtype=object)

Besides simply *looking* at these data, we can use these keys to filter our .vcf based on certain criteria. Let's start by dropping all sites below a certain quality score. First, though, how many SNPs do we actually have? We can evaluate this by taking the first entry from the `shape` attribute of the array: 

In [18]:
s['calldata/GT'].shape[0]

120150

Let's practice by selecting a single scaffold from the vcf file. First, we'll generate a list of all the scaffolds in the .vcf header:

In [19]:
scaffolds = s['variants/CHROM']
scaffolds

array([u'scaffold_6', u'scaffold_6', u'scaffold_6', ..., u'scaffold_561',
       u'scaffold_561', u'scaffold_561'], dtype=object)

Next, we'll generate a boolean array, indicating whether each site in the .vcf is found on scaffold_6 or not:

In [21]:
scaffold_6 = (scaffolds[0:]=='scaffold_6')
scaffold_6

array([ True,  True,  True, ..., False, False, False])

As confirmation, we can check that the length of this array is equivalent to the number of sites in the .vcf:

In [22]:
len(scaffold_6)

120150

Next, let's take the actual genotype calls from the vcf-turned-numpy-array and turn it into the GenotypeArray object we're familiar with:

In [24]:
gt = s['calldata/GT']
gt = allel.GenotypeArray(gt)
gt

<GenotypeArray shape=(120150, 18, 2) dtype=int8>
0/0 0/0 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/0 1/1
1/1 1/1 1/1 1/1 1/1 ... 1/1 ./. 1/1 1/1 1/1
0/1 0/0 1/1 0/0 0/1 ... 0/0 1/1 0/1 0/1 0/1
...
0/1 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0
0/1 0/1 0/1 1/1 1/1 ... 0/1 1/1 1/1 0/1 0/1
0/1 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0

**QUESTION 1:** How many samples do we have in this file?

We can then subset this by using the `compress()` function in numpy, with the `scaffold_6` object as our designated slice, applied to the first (or 0th, because of Python's dumb indexing) axis of the data, meaning the rows / sites:

In [27]:
gt_scaffold_6 = gt.compress(scaffold_6, axis=0)
gt_scaffold_6

<GenotypeArray shape=(15639, 18, 2) dtype=int8>
0/0 0/0 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/0 1/1
1/1 1/1 1/1 1/1 1/1 ... 1/1 ./. 1/1 1/1 1/1
0/1 0/0 1/1 0/0 0/1 ... 0/0 1/1 0/1 0/1 0/1
...
1/1 1/1 ./. 1/1 1/1 ... 1/1 0/0 1/1 0/0 1/1
1/1 1/1 ./. 1/1 1/1 ... 1/1 0/0 1/1 0/0 1/1
0/0 ./. 0/0 0/1 0/0 ... 0/0 0/0 0/0 0/1 0/0

This tells us 15638 sites come from scaffold 6. Next, let's filter sites based depth of coverage (DP). We'll first check the minimum and maximum depth values: 

In [29]:
dp_scores = (s['calldata/DP'])
dp_scores_nona = dp_scores[dp_scores >= 0] # this excludes negative values (missing data)
np.amin(dp_scores_nona)

1

In [30]:
np.amax(dp_scores_nona)

250

In otherwords, each indivdual has as few as 1 and as many as 250 reads. To reduce the influence of genotyping errors and paralogs, let's arbitrarily set a minimum depth of coverage of 5x, and a maximum of 100x. We'll generate two boolean arrays, and then find their intersection with `numpy's` `logical_and()` function:

In [32]:
pass_DP_min = dp_scores[:]>4
pass_DP_min = np.all(pass_DP_min, axis=1)
pass_DP_max = dp_scores[:]<100
pass_DP_max = np.all(pass_DP_max, axis=1)
pass_DP = np.logical_and(pass_DP_min, pass_DP_max)

This again generates a one-dimensional array equal to the total number of SNPs:

In [33]:
len(pass_DP)

120150

How many sites does this drop?

In [34]:
gt_dp = gt.compress(pass_DP, axis=0)
gt_dp

<GenotypeArray shape=(10765, 18, 2) dtype=int8>
0/0 0/1 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 1/1 0/0 0/0 0/1
0/0 0/0 0/0 0/1 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/1 1/1 1/1 0/1 1/1 ... 0/1 1/1 1/1 0/1 0/1
0/1 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0
0/1 0/1 0/1 1/1 1/1 ... 0/1 1/1 1/1 0/1 0/1

Roughly 10000!

We can use the sample `logical_and` function to select which of these sites are from scaffold 6:

In [520]:
passing = np.logical_and(pass_Q, scaffold_6)

In [521]:
final = gt.compress(passing, axis=0)

In [522]:
final

<GenotypeArray shape=(14001, 18, 2) dtype=int8>
0/0 0/0 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/0 1/1
1/1 1/1 1/1 1/1 1/1 ... 1/1 ./. 1/1 1/1 1/1
0/1 0/0 1/1 0/0 0/1 ... 0/0 1/1 0/1 0/1 0/1
...
0/0 0/0 ./. 0/0 0/1 ... 0/0 1/1 0/0 1/1 0/0
1/1 1/1 ./. 1/1 1/1 ... 1/1 0/0 1/1 0/0 1/1
1/1 1/1 ./. 1/1 1/1 ... 1/1 0/0 1/1 0/0 1/1

**QUESTION 2**: What is the highest quality score in our .vcf file?

In [37]:
q_scores = s['variants/QUAL']
np.amax(q_scores)

213950.0

Next, we'll filter for missing data, as more than a few missing calls per locus will prevent us from  running analyses we're interested in. This is a little more complicated given we can have two different dimensions of missing data: per individual, and per site. For a downstream analysis, we want to drop all sites with any missing data. First, we'll create a 2D boolean array indicating whether a given site for a given individual is missing. As a reminder, missing data is coded by a negative value here. Counting missing data per site results in either a 0 or a 1, based on previous filtering:  

In [38]:
count_MD = gt.count_missing(axis=1)
np.unique(count_MD)

array([0, 1])

In [39]:
pass_MD = count_MD==0
np.unique(pass_MD, return_counts=True)

(array([False,  True]), array([79718, 40432]))

As before, this becomes a boolean with information for each site:

Lastly, let's filter sites based on their minor allele frequency (MAF). To minimize the influence of sequencing errors, we're going to drop all sites at a frequency lower than 2/number of samples: 

In [47]:
maf_cutoff = 0.25
maf_cutoff

0.25

Next, we're going to extract allele frequency information from the .vcf, tell numpy to ignore warnings triggered by NAs in this array, and generate the pass / fail boolean we're familiar with:

In [48]:
AF = s['variants/AF']
AF
np.warnings.filterwarnings('ignore')
pass_AF = (AF[:,0]>maf_cutoff)

Let's apply this filter, our missing data filter, and our depth filter to the full datset for downstream analyses:

In [49]:
passing = np.logical_and(pass_DP, pass_MD, pass_AF)
final = gt.compress(passing, axis=0)
final

<GenotypeArray shape=(10673, 18, 2) dtype=int8>
0/0 0/1 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 1/1 0/0 0/0 0/1
0/0 0/0 0/0 0/1 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/1 1/1 1/1 0/1 1/1 ... 0/1 1/1 1/1 0/1 0/1
0/1 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0
0/1 0/1 0/1 1/1 1/1 ... 0/1 1/1 1/1 0/1 0/1

This leaves us with 4041 diploid SNPs for 18 individuals.

**QUESTION 3**: How many SNPs would we have if applied an MAF filter of 0.25?

### Principal Component Analysis

Introduced by Patterson, Price, and Reich in 2006, PCA has quickly become a ubiquitious tool for exploring genomic data. scikit-allel implements a fast version of this with its `allel.pca()` function. Because of a bug I was unable to fix by presentation time, we're going to use a second .vcf file I filtered previously:

In [50]:
s_alt = allel.read_vcf("syma_alt.vcf.gz", fields='*')

We'll turn it into a GenotypeArray, and then turn our array into a two-dinmensional matrix, where each cell is the number of non-reference alleles present:

In [51]:
gt_alt = s_alt['calldata/GT']
gt_alt = allel.GenotypeArray(gt_alt)
gt_alt

<GenotypeArray shape=(87923, 35, 2) dtype=int8>
0/0 1/1 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/1 0/1 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0
0/1 0/1 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/1 0/0
...
0/1 0/1 0/1 0/1 0/1 ... 0/1 0/1 0/1 0/1 0/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/1 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/1 0/0 0/0 0/0 0/0

**QUESTION 4:** What is the mean observed heterozygosity of these data?

Next, we'll run the function, using Patterson's (2006) scaling method. This will generate two objects: an array of coordinates for the 10 PC axes we designated, and an object of model parameters. 

In [54]:
gn = gt_alt.to_n_alt()
np.shape(gn)

(87923, 35)

In [59]:
coords1, model1 = allel.pca(gn, n_components=2, scaler='patterson')

Because I hate Python's plotting libraries, we're going to export these data for visualization in R, writing the PC axes to a .csv file with numpy's savetxt() function:

In [60]:
cols = "PC1,PC2"
np.savetxt('pca.csv', coords1, delimiter=',',header=cols)

The model object itself has useful information like the number of components, the scaler we used, and the variance explained by each component, which we can look at using the `__dict__.keys()` function:

In [61]:
model1.__dict__.keys()

['components_',
 'scaler',
 'explained_variance_ratio_',
 'explained_variance_',
 'n_components',
 'scaler_',
 'copy']

We're most interested in `explained_variance_ratio_`:

In [600]:
model1.explained_variance_ratio_

array([0.08543434, 0.04809955, 0.04058983, 0.03549726, 0.0336217 ,
       0.03301791, 0.03114965, 0.03057619, 0.03008257, 0.0295906 ],
      dtype=float32)

About 8.5% of genotypic variation is explained by the first principal component!